### Setup

In [5]:
from google.cloud import pubsub_v1
import json
import requests
import mysql.connector
import os
from datetime import datetime

### Publiser

In [17]:
topic_path = 'projects/auditpulse/topics/deployment-request-queue'
data = {
         'run_id': 81102,
         'user_id':1023,
         'central_index_key': '1249',
         'company_name':"Some Inc.",
         'ticker':'SINC',
         'year':2025
        }

jsonstring = json.dumps(data)
jsonstring = jsonstring.encode('utf-8')
publisher = pubsub_v1.PublisherClient()
future = publisher.publish(topic_path, jsonstring)

### Subscriber

In [18]:
def sub_callback(message):
    print(f'Data Received: {message}')
    global msg
    msg = message
    message.ack()
msg = None
subscriber_path = 'projects/auditpulse/subscriptions/deployment-request-queue-sub'
subscriber = pubsub_v1.SubscriberClient()
subscriber_future = subscriber.subscribe(subscriber_path, callback=sub_callback)
result = None
with subscriber:
    try:
        result = subscriber_future.result(timeout=5)
    except:
        subscriber_future.cancel()
        result = subscriber_future.result()

Data Received: Message {
  data: b'{"run_id": 81102, "user_id": 1023, "central_index_...'
  ordering_key: ''
  attributes: {}
}


In [19]:
msg

Message {
  data: b'{"run_id": 81102, "user_id": 1023, "central_index_...'
  ordering_key: ''
  attributes: {}
}

In [20]:
json.loads(msg.data)

{'run_id': 81102,
 'user_id': 1023,
 'central_index_key': '1249',
 'company_name': 'Some Inc.',
 'ticker': 'SINC',
 'year': 2025}

### Update Table

In [6]:
mysql_conn = mysql.connector.connect(
    host='34.46.191.121',
    port=3306,
    user='root',
    database='auditpulse',
    password=os.getenv('MYSQL_GCP_PASS')
)

In [9]:
mysql_cursor = mysql_conn.cursor()

query = """
INSERT INTO runs 
(run_id, run_at, user_id, cik, ticker, company_name, year, status)
VALUES 
(%s, %s, %s, %s, %s, %s, %s, %s)
"""

# Current timestamp for run_at
current_time = datetime.now()

# Values to insert
values = (
    "81102",
    current_time,
    "1023",
    "320193",
    "AAPL",
    "Apple Inc.",
    2025,
    "queued"
)

mysql_cursor.execute(query,values)
mysql_conn.commit()

### Request

In [12]:
data = {
         'run_id': "81102",
         'user_id':"1023",
         'central_index_key': '320193',
         'company_name':"AAPL",
         'ticker':'Apple Inc.',
         'year':2025
        }

jsonstring = json.dumps(data)
# jsonstring = jsonstring.encode('utf-8')

In [14]:
endpoint = 'http://127.0.0.1:5000/generate'
response = requests.post(endpoint, json={'message':{'data':jsonstring}})

In [30]:
response.text

'{\n  "message": "Report generated!",\n  "status": "Success!"\n}\n'